In [ ]:
from openai import OpenAI
# client = OpenAI()
import csv
import pandas as pd
import psycopg2
from datetime import datetime
import random
from tenacity import retry, stop_after_attempt, wait_random_exponential
import json
import re
import ollama
import os
import time
import openai
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import openai
default_credential = DefaultAzureCredential()
api_type = "azure"
api_base =
api_version =
mgb_openai_client = openai.AzureOpenAI(api_version=api_version, azure_endpoint=api_base, api_key = '')


In [ ]:
combined_data = pd.read_csv('../data/final_patient_data_table_no_med.csv', sep=',')
subset_empi = pd.read_csv('../data/corrected_everything.csv')

combined_data['empi'] = combined_data['empi'].astype(str)
subset_empi['empi'] = subset_empi['empi'].astype(str)

In [ ]:
subset_data = combined_data[combined_data['empi'].isin(subset_empi['empi'])]

In [ ]:
print(subset_data)
print(subset_data.columns)

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return mgb_openai_client.chat.completions.create(**kwargs)

import tiktoken

enc = tiktoken.encoding_for_model("gpt-4o")

## Go through the entire records
def query_MGB_API(empi, medical_summaries, dates, model):
    interactions = []
    error_list = []
    system_message = "You are a knowledgeable and elite professional healthcare specialist, able to answer complicated and medical related questions from clinical notes. When answering questions, answer based on your knowledge and the provided patient notes. The question will be answered by yes or no format. After answering the question, give thorough explanations and reasons for your decision. Include specific information from the notes and cite the report number when it\'s helpful. Make your answer evidence based."

    # Initialize the prior decision as "NO"
    prior_status = "NO"
    prior_reason = None

    # Initialize error_messages as an empty list for each iteration
    error_messages = []

    # Include prior status in the prompt
    messages = [
        {"role": "system", "content": system_message},
    ]

    concat_summary = []

    for medical_summary, date in zip(medical_summaries, dates):
        concat_summary.append(f"\n\n\n\nDocument date: {date}\n\n{medical_summary}")

    while len(enc.encode("".join(concat_summary))) > 120000:
        concat_summary.pop(0)

    messages.append({"role": "user", "content": "".join(concat_summary)})

    # P1
    messages.append({"role": "user", "content": ("Based on above documents, does the patient have dementia? Determine if there is an explicit and confirmed diagnosis of dementia specifically for this patient (not a family member or relative). Disregard mentions of cognitive complaints, medications, treatment trials (e.g., rivastigmine or Aricept), or indirect assessments (such as work-ups or evaluations) that do not explicitly state a dementia diagnosis. Consider more recent notes with more importance than older ones, and allow newer notes to negate older diagnosis if dementia information is conflicting.\n\nIf results of cognitive tests like MMSE, MOCA, or Mini-cog are mentioned, interpret them as following:\nMMSE: 24 and higher: Normal cognition, no dementia. 19-23: mild dementia. 10-18: moderate dementia. 9 and lower: severe dementia.\n\nMOCA: 26 or above: normal. 18-25: mild cognitive impairment. 10-17: moderate cognitive impairment. Less than 10: severe cognitive impairment.\n\nMini-cog: If 0 of out 3 words are recalled, it is a positive screen for dementia, regardless of the clock-drawing. If all 3 words are recalled, it is immediately a negative screen for dementia, regardless of the clock-drawing. If the patient recalls 1-2 words, then the clinician needs to further refer to the clock-drawing: If there is 1-2 words (of out 3) for recall, with a normal clock, it is a negative screen for dementia. If there is 1-2 words (of out 3) for recall, with an abnormal clock, it is a positive screen for dementia. Respond in the following format:"
         "{\n"
         "  \"Dementia_Status\": \"YES\" or \"NO\",\n"
         "  \"Reason\": \"Brief explanation of the conclusion.\",\n"
         "}"
    )})

    # P2
    # messages.append({"role": "user", "content": (
    #         f"As of the most recent available documentation, does the patient have dementia?\n"
    #         f"Carefully review the timeline of clinical documentation. Determine whether there is an explicit and confirmed diagnosis of dementia specifically for this patient (not a family member or relative).\n"
    #         "Pay close attention to when information was recorded — prioritize more recent notes and diagnoses, but consider earlier entries to evaluate progression over time.\n"
    #         "Disregard:\n"
    #         "- Mentions of cognitive complaints alone\n"
    #         "- Medications or treatment trials (e.g., rivastigmine, Aricept) without a confirmed diagnosis\n"
    #         "- Indirect assessments (e.g., \"work-up for dementia\" or \"referred for evaluation\") unless they result in a clear diagnosis\n"
    #         "Disregard mentions of cognitive complaints, medications, treatment trials (e.g., rivastigmine or Aricept), or indirect assessments (such as work-ups or evaluations) that do not explicitly state a dementia diagnosis. \n"
    #         "\n\nRespond in the following format:\n\n"
    #         "{\n"
    #         "  \"Dementia_Status\": \"YES\" or \"NO\",\n"
    #         "  \"Reason\": \"Brief explanation of the conclusion.\",\n"
    #         "}"
    #     )})

    # P3
#     messages.append({"role": "user", "content": """Step 1: Analyze the Timeline of Documentation
# Review the complete timeline of clinical notes and diagnoses to determine if a diagnosis of dementia was explicitly made for this patient (not for a family member).
# Prioritize more recent clinical notes and diagnoses, ensuring they align with and confirm earlier findings. If there is only one dementia-related diagnosis code (e.g., Alzheimer’s disease or related dementia), subsequent documentation must continue to support or confirm the diagnosis for it to be valid.
#
#
# Step 2: Assess Supporting Evidence
# Consider Cognitive Test Scores: Include scores from cognitive assessments (e.g., MoCA, MMSE). Low scores alone do not confirm dementia but, if consistently impaired over time or paired with a diagnosis, they strengthen the case.
# Evaluate Functional Status: If the patient is described as functionally independent or improving and there is no corroborating evidence of progressive decline, carefully reassess the validity of the earlier dementia diagnosis.
# Transitions in Diagnosis: If earlier records list dementia but newer documentation clarifies mild cognitive impairment (MCI) or indicates improvement, favor the most recent clinically-supported status only if it contradicts earlier findings.
#
#
# Step 3: Confirm Continuity of Evidence
# Explicitly check whether the latest clinical documentation reaffirms an earlier dementia-related diagnosis. If the most recent evidence does not explicitly confirm or support the earlier diagnosis, reassess its validity.
#
#
# Step 4: Disregard Irrelevant Mentions
# Ignore the following unless explicitly tied to a clinical confirmation of dementia:
# Memory concerns or subjective complaints without diagnosis.
# Medications (e.g., donepezil, rivastigmine) unless paired with a documented diagnosis.
# Phrases like “rule out dementia,” “referred for cognitive workup,” or “possible Alzheimer’s” unless subsequent documentation confirms diagnosis.
# Isolated billing codes or problem list entries unless supported by clinical notes or functional assessments.
#
#
# Step 5: Answer Classification
# Provide your answer based on the following criteria:
#
# "Yes" if:
# There is an explicit dementia diagnosis in the most recent clinical documentation and it aligns with or confirms earlier findings.
# The diagnosis is supported by clinical observations, cognitive testing, or evidence of progressive functional decline.
# "No" if:
# There is no confirmed dementia diagnosis in the most recent documentation.
# The latest evidence indicates MCI or contradicts earlier mentions of dementia.
# The earlier dementia diagnosis is unsupported by subsequent clinical findings, particularly when there is only one dementia-related diagnosis code.
#
# Respond in the following json format:
# {
#     "Dementia_Status: "YES" or "NO",
#     "Reason: "Brief explanation of the conclusion."
# }
# """})



    interaction_record = {
        "empi": empi,
        "date": date,
        "messages_sent": messages,
        "model_response": None,
        "Dementia_Status": None,
        "Status_Change": None,
        "Reason": None,
        "Relevant_Text": None,
        "error_message": None,
        "processing_time": None  # Add processing_time field
    }

    try:
        if model == "gpt-4o":
            start_time = time.time()  # Start timing before API call
            # Send the completion request with backoff handling
            response = completion_with_backoff(
                model="gpt-4o-2",
                messages=messages,
                temperature=0,
            )

            end_time = time.time()  # End timing after receiving response
            processing_duration = end_time - start_time  # Calculate duration
            interaction_record["processing_time"] = processing_duration  # Store duration

            response_text = response.choices[0].message.content.strip()
            if response_text[:3] == '```':
                # The model responded in ```json\n{stuff}\n``` format instead of {stuff}
                # trim it
                response_text = response_text[8:-4]


            response_json = json.loads(response_text)
        else:
            start_time = time.time()
            response = ollama.chat(
                model=model,
                messages=messages,
                format='json',
                options={'temperature': 0}
            )

            end_time = time.time()  # End timing after receiving response
            processing_duration = end_time - start_time  # Calculate duration
            interaction_record["processing_time"] = processing_duration  # Store duration

            response_text = response.message.content.strip()
            print(response_text)
            response_json = json.loads(response_text)

        interaction_record["model_response"] = response_text

        # Parse the response
        interaction_record["Dementia_Status"] = response_json.get("Dementia_Status")
        interaction_record["Status_Change"] = response_json.get("Status_Change")
        interaction_record["Reason"] = response_json.get("Reason")
        interaction_record["Relevant_Text"] = response_json.get("Relevant_Text")

        # Update prior status and reason for the next summary
        prior_status = interaction_record["Dementia_Status"]
        prior_reason = interaction_record["Reason"]

        interactions.append(interaction_record)

    except Exception as e:
        interaction_record["error_message"] = f"Error: {str(e)}"
        interactions.append(interaction_record)

    return interactions

def run_real_data(df, empis, model, output_file):
    # Track EMPIs that have already been processed
    processed_empi = set()

    # Check if the output file exists and read existing processed EMPIs
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file)
        processed_empi.update(existing_df['empi'].unique())
        print(f"Loaded {len(processed_empi)} processed EMPIs from existing file.")

    # Initialize CSV file with headers if not already present
    header_written = os.path.exists(output_file)

    # Process each patient's records individually, skipping already processed EMPIs
    for empi, group in df.groupby('empi'):
        if empi not in empis['empi'].values or empi in processed_empi:
            print(f"Skipping already processed EMPI: {empi}")
            continue  # Skip if EMPI is not in the subset list or already processed

        # Sort records by date for each patient
        group = group.sort_values(by='record_date')
        medical_summaries = group['medical_summary'].tolist()
        # get rid of medications from structured data
        dates = group['record_date'].tolist()
        print('A total of records:', len(medical_summaries))

        # Retrieve human label and final diagnosis date for comparison
        human_label_info = empis[empis['empi'] == empi].iloc[0]
        final_label = human_label_info['label']

        # Query API for each patient's records
        interaction_records = query_MGB_API(empi, medical_summaries, dates, model)

        # Add human label information to each interaction record
        for record in interaction_records:
            record.update({
                "final_label": final_label,
            })

        # Convert each patient's interaction_records to a DataFrame
        patient_interactions_df = pd.DataFrame(interaction_records)

        # Save to CSV with append mode, writing header only if it hasn't been written
        if not header_written:
            patient_interactions_df.to_csv(output_file, index=False, mode='w')
            header_written = True
        else:
            patient_interactions_df.to_csv(output_file, index=False, mode='a', header=False)

        print(f"Results for EMPI {empi} saved.")

In [ ]:
output_file = 'no med keyword deepseek p1.csv'
# Call the function with the output file path as an argument
run_real_data(subset_data, subset_empi, 'deepseek-r1:70b', output_file) # 'gpt-4o' or ollama models

In [ ]:
print("Done")

In [ ]:
check_empi = pd.read_csv(output_file)

In [ ]:
print(len(check_empi['empi']))

In [ ]:
a = pd.read_csv("merged concat summary gpt4o baoren prompt.csv")